# Getting started with the EB-NeRD

In [ ]:
import polars as pl

In [ ]:
df_articles = pl.read_parquet("/kaggle/input/recommender-systems-challenge-2024/ebnerd_demo/articles.parquet", columns = ["article_id", "topics"]).sample(fraction= 0.7)
df_behaviors = pl.read_parquet("/kaggle/input/recommender-systems-challenge-2024/ebnerd_demo/train/behaviors.parquet", columns = ["user_id", "article_ids_inview"]).sample(fraction = 0.7)
df_history = pl.read_parquet("/kaggle/input/recommender-systems-challenge-2024/ebnerd_demo/train/history.parquet", columns = ["user_id", "article_id_fixed"]).sample(fraction = 0.7)

In [ ]:
train_ds = df_behaviors.explode("article_ids_inview")
train_ds = train_ds.rename({"article_ids_inview": "article_id"})
train_ds

In [ ]:
train_ds = train_ds.join(df_articles, on= "article_id")
train_ds

In [ ]:
df_history = df_history.explode("article_id_fixed")
df_history = df_history.rename({"article_id_fixed": "article_id"})
df_history

In [ ]:
df_history = df_history.join(df_articles, on= "article_id")
df_history = df_history.drop("article_id")
df_history

In [ ]:
train_ds

In [ ]:
def jaccard_similarity(list1,list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    if len(set1) == 0 or len(set2) == 0:
        return 0
    similarity = intersection / union
    return similarity

In [ ]:
# user_id article_id topic1 topic2
merged_df = train_ds.join(df_history, on='user_id', suffix='_2')

In [ ]:
del df_articles
del df_behaviors
del df_history

In [ ]:
merged_df

In [ ]:
merged_df = merged_df.with_columns(
    merged_df.map_rows(lambda row: jaccard_similarity(row[2],row[3]))
)

In [ ]:
merged_df

In [ ]:
# raggruppo per user_id e articolo e faccio media
result_df = (
    merged_df.select(["user_id", "article_id", "map"])
    .group_by(["user_id", "article_id"])
    .agg(pl.mean("map").alias("avg_jaccard_similarity"))
)



In [ ]:
result_df

In [ ]:
df1 = result_df.join(train_ds, on=["user_id", "article_id"], how="left")

In [ ]:
df1